In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 30233513
paper_name = 'grosjean_blaudez_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table_2_Global Deletome Profile of Saccharomyces cerevisiae Exposed to the Technology-Critical Element Yttrium.XLSX', 
                              sheet_name='Supp Table 2', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 333 x 10


In [7]:
original_data.head()

,SGD ID,ORF,Yttrium,ORF name,ORF Name description,Description,ORF type,H. sapiensorthologs,Unnamed: 8,Human disease
0,S000000240,YBR036C,HS,CSG2,Calcium Sensitive Growth,Endoplasmic reticulum membrane protein; requir...,Verified,-,-,-
1,S000005977,YPL056C,HS,LCL1,Long Chronological Lifespan,Putative protein of unknown function; deletion...,Uncharacterized,-,-,-
2,S000001626,YKL143W,HS,LTV1,Low Temperature Viability,Component of the GSE complex; GSE is required ...,Verified,-,-,-
3,S000005978,YPL057C,HS,SUR1,SUppressor of Rvs161 and rvs167 mutations,Mannosylinositol phosphorylceramide (MIPC) syn...,Verified,-,-,-
4,S000006367,YPR163C,HS,TIF3,Translation Initiation Factor,Translation initiation factor eIF-4B; contains...,Verified,"eukaryotic translation initiation factor 4B, 4H","EIF4B, EIF4H",-


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [SGD ID, ORF, Yttrium, ORF name, ORF Name description, Description, ORF type, H. sapiensorthologs, Unnamed: 8, Human disease, orf]
Index: []


In [12]:
original_data['Yttrium'].unique()

array(['HS', 'MS', 'LS', 'LR', 'MR', 'HR'], dtype=object)

In [15]:
data_dict = {'HS': -3, 'MS': -2, 'LS': -1, 'LR': 1, 'MR': 2, 'HR': 3}

In [16]:
original_data['data'] = original_data['Yttrium'].apply(lambda x: data_dict[x])

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(333, 1)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [22227]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,22227
data_type,value
orf,
YAL016W,1
YAL020C,1
YAL053W,1
YAL058W,1
YAR018C,1


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22227
,data_type,value
gene_id,orf,
14,YAL016W,1
18,YAL020C,1
49,YAL053W,1
53,YAL058W,1
68,YAR018C,1


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22227          
data_type       value    valuez
gene_id orf                    
14      YAL016W     1  1.954177
18      YAL020C     1  1.954177
49      YAL053W     1  1.954177
53      YAL058W     1  1.954177
68      YAR018C     1  1.954177

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
# from IO.save_data_to_db3 import *

In [36]:
# save_data_to_db(data_all, paper_pmid)